In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
import nest_asyncio
import warnings

_ = load_dotenv(find_dotenv())
nest_asyncio.apply()
warnings.filterwarnings('ignore')

In [127]:
from llama_index.core import Settings
from llama_index.core.tools import FunctionTool
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.embeddings.bedrock import BedrockEmbedding

import yfinance as yf
import pandas as pd

from typing import Optional, Literal, List, Union, Tuple
from datetime import datetime as dt

In [152]:
class FundamentalAnalyst:
    def __init__(self, ticker: str):
        """Initialize the fundamental analyst tool"""
        self.ticker = ticker
        
        ## Financial statements
        self.balance_sheet = yf.Ticker(self.ticker).balancesheet.transpose()
        self.income_statement = yf.Ticker(self.ticker).financials.transpose()
        self.cashflow_statement = yf.Ticker(self.ticker).cashflow.transpose()
        self.actions = yf.Ticker(self.ticker).actions
        
        ## Filter data from yahoo finance
        self.data = yf.Ticker(self.ticker).history(period="5y").tz_localize(None)
        self.dates = [date.date() for date in self.balance_sheet.index]
        self.data = self.data[self.data.index.isin(self.dates)]
        
        ## Initialize dummy dataframe
        self._df = pd.DataFrame()
    
    def get_income_magic_ratios(self):
        """Returns magic ratios from income statement"""
        self._df['Gross Margin (%)'] = self.income_statement['Gross Profit']*100/self.income_statement['Total Revenue']
        self._df['Net Margin (%)'] = self.income_statement['Net Income'] * 100 / self.income_statement['Total Revenue']
    
    def get_roa(self):
        """Returns 'return on asset' ratio"""
        self._df['ROA (%)'] = self.income_statement['Net Income'] * 100 / self.balance_sheet['Total Assets']
    
    def get_roe(self):
        """Returns 'return on equity' ratio"""
        self._df['ROE (%)'] = self.income_statement['Net Income'] * 100 / self.balance_sheet['Stockholders Equity']
        
    def get_current_ratio(self):
        """Returns current ratio """    
        self._df['Current Ratio'] = self.balance_sheet['Current Assets'] / self.balance_sheet['Current Liabilities']
    
    def get_quick_ratio(self):
        """Returns quick ratio"""
        self._df['Quick Ratio'] = (self.balance_sheet['Current Assets'] - self.balance_sheet['Inventory'])/self.balance_sheet['Current Liabilities']
    
    def get_debt_to_equity(self):
        """Returns debt-to-equity ratio"""
        self._df['debt_to_equity'] = self.balance_sheet['Total Liabilities Net Minority Interest'] / self.balance_sheet['Stockholders Equity']
    
    def get_debt_to_assets(self):
        """Returns debt-to-asset ratio"""
        self._df['debt_to_asset'] = self.balance_sheet['Total Liabilities Net Minority Interest'] / self.balance_sheet['Total Assets']
    
    def get_pe_ratio(self):
        """Returns P/E ratio"""
        self._df['P/E Ratio'] = self.data['Close']/self.income_statement['Basic EPS']
    
    def get_price_to_book_ratio(self):
        """Returns price-to-book ratio. The price to book ratio reflects
        the value that the market participants attach to a company's equity relative
        to the book value of its equity. An undervalued stock is a stock with a P/B ratio < 1."""
        self._df['P/B Ratio'] = self.data['Close']/((self.balance_sheet['Total Assets'] - self.balance_sheet['Total Liabilities Net Minority Interest'])/self.data['Volume'])
    
    def get_price_to_sales_ratio(self):
        """Computes price-to-sales ratio. This ratio shows how much investors are willing to pay
        per dollar of sales for a stock."""
        self._df['P/S Ratio'] = self.data['Close']/(self.balance_sheet['Stockholders Equity']/self.data['Volume'])
    
    def analyse(self):
        """Super function that calls all other methods within class"""
        self.get_income_magic_ratios()
        self.get_current_ratio()
        self.get_roa()
        self.get_roe()
        self.get_quick_ratio()
        self.get_debt_to_assets()
        self.get_debt_to_equity()
        self.get_pe_ratio()
        self.get_price_to_book_ratio()
        self.get_price_to_sales_ratio()
        return self._df.iloc[:-1]
    
    @property
    def df(self):
        """For debugging"""
        return self._df

In [153]:
fa = FundamentalAnalyst(ticker="ilmn")
df = fa.analyse()

In [154]:
df

,Gross Margin (%),Net Margin (%),Current Ratio,ROA (%),ROE (%),Quick Ratio,debt_to_asset,debt_to_equity,P/E Ratio,P/B Ratio,P/S Ratio
2023-12-31,60.923623,-25.777087,1.661783,-11.482544,-20.208877,1.287898,0.431807,0.759965,NaN,NaN,NaN
2022-12-31,64.834206,-96.073298,1.284169,-35.945152,-66.737384,1.079336,0.461394,0.856645,NaN,NaN,NaN
2021-12-31,69.686257,16.836058,2.482159,5.007557,7.094972,2.087832,0.29421,0.416853,72.993653,0.016596,0.016596
2020-12-31,68.014819,20.253165,3.603698,8.648649,13.975288,3.304662,0.381147,0.615893,80.339772,0.040484,0.040484


In [175]:
def evaluate_fundamentals(ticker: str):
    """
    This tool is used for fundamental analysis of stock prices.
    
    The aim of fundamental analysis is to identify potentially undervalued stocks. We
    do this by ascertaining the company fundamentals and market signals:
    
    1. Gross margin (%): The ratio of the Gross profit / Total income. This informs about whether
    the company is making a profit from its sales in the market.
    2. Net Margin (%): The ratio of net proft / total income. This informs on whether the company
    makes a profit after including operating costs. 
    3. Current Ratio: The ratio of current assets/current liabilities. This informs on whether
    the company has more assets on hand than liabilities on hand.
    4. Quick Ratio: The ratio of current assets less inventory / current liabilities. This gives a
    more in-depth picture on whether the company has more assets than liabilities on hand.
    5. Return on Assets (%): Ratio of net income /total assets. Shows how much profit a company is
    able to generate from its assets.
    6. Return on Equity (%): Ratio of net income against shareholder equity. Shows how efficiently
    a company is generating income from the equity investments of shareholders.
    7. Debt to assets (%): Ratio of a company's total liabilities against total assets. Shows the
    degree to which a company has used debt to finance its assets
    8. Debt to equity (%): Ratio of a company's total liabilities to its shareholder equity. If the
    ratio is 1.5, it means the company is $1.50 in debt for every $1 of equity.
    9. P/E ratio: Ratio of share price to earnings per share. Provides an indication to whether a
    stock at its current market price is expensive (high P/E ratio) or cheap (low P/E ratio). 
    10. P/B ratio: Ratio of share price to book value (calculated as total assets - total liabilities/volume).
    A P/B ratio of less than 1 suggests a stock is undervalued.
    11.P/S ratio: Ratio of share price to (stockholders equity/volume). Indicates how much 
    investors are willing to pay. If P/S ratio is low, the stock is likely overvalued and investors
    prefer paying less.
    """
    fa = FundamentalAnalyst(ticker=ticker)
    df = fa.analyse()
    df = df.fillna(0) 
    return df

In [176]:
fa_tool = FunctionTool.from_defaults(evaluate_fundamentals)

In [177]:
Settings.llm = BedrockConverse(
    model = "anthropic.claude-3-haiku-20240307-v1:0",
    aws_access_key_id = os.environ["AWS_ACCESS_KEY"],
    aws_secret_access_key = os.environ["AWS_SECRET_ACCESS_KEY"],
    region_name = os.environ["AWS_DEFAULT_REGION"]
)
Settings.embed_model = BedrockEmbedding(
    model = "amazon.titan-embed-text-v1",
    aws_access_key_id = os.environ["AWS_ACCESS_KEY"],
    aws_secret_access_key = os.environ["AWS_SECRET_ACCESS_KEY"],
    aws_region_name = os.environ["AWS_DEFAULT_REGION"]
)

In [180]:
from llama_index.core.agent import (
    FunctionCallingAgentWorker,
    AgentRunner
)
from IPython.display import display, Markdown

agent_worker = FunctionCallingAgentWorker.from_tools(
    tools = [fa_tool],
    llm = Settings.llm,
    verbose = True)
agent = AgentRunner(agent_worker=agent_worker)

In [181]:
query = "Conduct a fundamental analysis on Illumina shares. Should I buy Illumina shares?"
response = agent.chat(query)

display(Markdown(f"<b>{response}</b>"))

Added user message to memory: Conduct a fundamental analysis on Illumina shares. Should I buy Illumina shares?
=== LLM Response ===
Okay, let's perform a fundamental analysis on Illumina (ticker: ILMN) to evaluate whether it's a good investment.
=== Calling Function ===
Calling function: evaluate_fundamentals with args: {"ticker": "ILMN"}
=== Function Output ===
            Gross Margin (%)  Net Margin (%)  Current Ratio    ROA (%)  \
2023-12-31         60.923623      -25.777087       1.661783 -11.482544   
2022-12-31         64.834206      -96.073298       1.284169 -35.945152   
2021-12-31         69.686257       16.836058       2.482159   5.007557   
2020-12-31         68.014819       20.253165       3.603698   8.648649   

              ROE (%)  Quick Ratio  debt_to_asset  debt_to_equity  P/E Ratio  \
2023-12-31 -20.208877     1.287898       0.431807        0.759965   0.000000   
2022-12-31 -66.737384     1.079336       0.461394        0.856645   0.000000   
2021-12-31   7.094972   

<b>Based on the fundamental analysis, here are my key observations on Illumina (ILMN):

1. Profitability Metrics:
   - Gross Margin has declined from 69.7% in 2021 to 60.9% in 2023, indicating lower profitability from sales.
   - Net Margin has swung from positive 16.8% in 2021 to negative 25.8% in 2023, showing the company is currently unprofitable.
   - Return on Assets (ROA) and Return on Equity (ROE) have both turned negative in the last two years, further confirming the company's poor profitability.

2. Liquidity Ratios:
   - Current Ratio has declined from 3.6 in 2020 to 1.7 in 2023, indicating the company may be struggling to meet short-term obligations.
   - Quick Ratio has also declined from 3.3 in 2020 to 1.3 in 2023, suggesting the company has less liquid assets to cover liabilities.

3. Leverage Ratios:
   - Debt-to-Asset ratio has increased from 0.29 in 2021 to 0.43 in 2023, meaning the company is relying more on debt to finance its operations.
   - Debt-to-Equity ratio has also risen from 0.42 in 2021 to 0.76 in 2023, indicating the company has a higher debt burden relative to shareholder equity.

4. Valuation Metrics:
   - P/E ratio has declined from 80.3 in 2020 to 0 in the last two years, likely due to the company's negative earnings.
   - P/B ratio has also declined from 0.04 in 2020 to 0 in the last two years, suggesting the stock may be undervalued.
   - P/S ratio has declined from 0.04 in 2020 to 0 in the last two years, indicating the stock may be trading at a discount to its sales.

Overall, the fundamental analysis paints a concerning picture for Illumina. The company's profitability, liquidity,</b>

## Export

In [1]:
%%writefile ../tools/fundamental_analysis_tools.py

from llama_index.core.tools import FunctionTool
import yfinance as yf
import numpy as np
import pandas as pd

class FundamentalAnalyst:
    def __init__(self, ticker: str):
        """Initialize the fundamental analyst tool"""
        self.ticker = ticker
        
        ## Financial statements
        self.balance_sheet = yf.Ticker(self.ticker).balancesheet.transpose()
        self.income_statement = yf.Ticker(self.ticker).financials.transpose()
        self.cashflow_statement = yf.Ticker(self.ticker).cashflow.transpose()
        self.actions = yf.Ticker(self.ticker).actions
        
        ## Filter data from yahoo finance
        self.data = yf.Ticker(self.ticker).history(period="5y").tz_localize(None)
        self.dates = [date.date() for date in self.balance_sheet.index]
        self.data = self.data[self.data.index.isin(self.dates)]
        
        ## Initialize dummy dataframe
        self._df = pd.DataFrame()
    
    def get_income_magic_ratios(self):
        """Returns magic ratios from income statement"""
        self._df['Gross Margin (%)'] = self.income_statement['Gross Profit']*100/self.income_statement['Total Revenue']
        self._df['Net Margin (%)'] = self.income_statement['Net Income'] * 100 / self.income_statement['Total Revenue']
    
    def get_roa(self):
        """Returns 'return on asset' ratio"""
        self._df['ROA (%)'] = self.income_statement['Net Income'] * 100 / self.balance_sheet['Total Assets']
    
    def get_roe(self):
        """Returns 'return on equity' ratio"""
        self._df['ROE (%)'] = self.income_statement['Net Income'] * 100 / self.balance_sheet['Stockholders Equity']
        
    def get_current_ratio(self):
        """Returns current ratio """    
        self._df['Current Ratio'] = self.balance_sheet['Current Assets'] / self.balance_sheet['Current Liabilities']
    
    def get_quick_ratio(self):
        """Returns quick ratio"""
        self._df['Quick Ratio'] = (self.balance_sheet['Current Assets'] - self.balance_sheet['Inventory'])/self.balance_sheet['Current Liabilities']
    
    def get_debt_to_equity(self):
        """Returns debt-to-equity ratio"""
        self._df['debt_to_equity'] = self.balance_sheet['Total Liabilities Net Minority Interest'] / self.balance_sheet['Stockholders Equity']
    
    def get_debt_to_assets(self):
        """Returns debt-to-asset ratio"""
        self._df['debt_to_asset'] = self.balance_sheet['Total Liabilities Net Minority Interest'] / self.balance_sheet['Total Assets']
    
    def get_pe_ratio(self):
        """Returns P/E ratio"""
        self._df['P/E Ratio'] = self.data['Close']/self.income_statement['Basic EPS']
    
    def get_price_to_book_ratio(self):
        """Returns price-to-book ratio. The price to book ratio reflects
        the value that the market participants attach to a company's equity relative
        to the book value of its equity. An undervalued stock is a stock with a P/B ratio < 1."""
        self._df['P/B Ratio'] = self.data['Close']/((self.balance_sheet['Total Assets'] - self.balance_sheet['Total Liabilities Net Minority Interest'])/self.data['Volume'])
    
    def get_price_to_sales_ratio(self):
        """Computes price-to-sales ratio. This ratio shows how much investors are willing to pay
        per dollar of sales for a stock."""
        self._df['P/S Ratio'] = self.data['Close']/(self.balance_sheet['Stockholders Equity']/self.data['Volume'])
    
    def analyse(self):
        """Super function that calls all other methods within class"""
        self.get_income_magic_ratios()
        self.get_current_ratio()
        self.get_roa()
        self.get_roe()
        self.get_quick_ratio()
        self.get_debt_to_assets()
        self.get_debt_to_equity()
        self.get_pe_ratio()
        self.get_price_to_book_ratio()
        self.get_price_to_sales_ratio()
        return self._df.iloc[:-1]
    
    @property
    def df(self):
        """For debugging"""
        return self._df

def evaluate_fundamentals(ticker: str):
    """
    This tool is used for fundamental analysis of stock prices.
    
    The aim of fundamental analysis is to identify potentially undervalued stocks. We
    do this by ascertaining the company fundamentals and market signals:
    
    1. Gross margin (%): The ratio of the Gross profit / Total income. This informs about whether
    the company is making a profit from its sales in the market.
    2. Net Margin (%): The ratio of net proft / total income. This informs on whether the company
    makes a profit after including operating costs. 
    3. Current Ratio: The ratio of current assets/current liabilities. This informs on whether
    the company has more assets on hand than liabilities on hand.
    4. Quick Ratio: The ratio of current assets less inventory / current liabilities. This gives a
    more in-depth picture on whether the company has more assets than liabilities on hand.
    5. Return on Assets (%): Ratio of net income /total assets. Shows how much profit a company is
    able to generate from its assets.
    6. Return on Equity (%): Ratio of net income against shareholder equity. Shows how efficiently
    a company is generating income from the equity investments of shareholders.
    7. Debt to assets (%): Ratio of a company's total liabilities against total assets. Shows the
    degree to which a company has used debt to finance its assets
    8. Debt to equity (%): Ratio of a company's total liabilities to its shareholder equity. If the
    ratio is 1.5, it means the company is $1.50 in debt for every $1 of equity.
    9. P/E ratio: Ratio of share price to earnings per share. Provides an indication to whether a
    stock at its current market price is expensive (high P/E ratio) or cheap (low P/E ratio). 
    10. P/B ratio: Ratio of share price to book value (calculated as total assets - total liabilities/volume).
    A P/B ratio of less than 1 suggests a stock is undervalued.
    11.P/S ratio: Ratio of share price to (stockholders equity/volume). Indicates how much 
    investors are willing to pay. If P/S ratio is low, the stock is likely overvalued and investors
    prefer paying less.
    """
    fa = FundamentalAnalyst(ticker=ticker)
    df = fa.analyse()
    df = df.fillna(0) 
    return df

def get_fa_tools():
    return FunctionTool.from_defaults(evaluate_fundamentals)

Overwriting fundamental_analysis_tools.py
